# Visualize original trajectories and their hashes in the grid scheme


In [21]:
import folium
import os
import sys

currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from hashed_similarities.grid_similarity import (
    generate_grid_hash_similarity_coordinates,
)

# Rome coordinates
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88
ROME_META_FILE = "../dataset/rome/output/META-50.txt"
RESOLUTION = 0.2
LAYERS = 2

# TODO: import from constants when merged
colors = [
    "red",
    "blue",
    "green",
    "purple",
    "orange",
    "darkred",
    "lightblue",
    "beige",
    "darkgreen",
    "cadetblue",
]

In [22]:
# Initialize a Folium map at a central point
map = folium.Map(
    location=[(R_MAX_LAT + R_MIN_LAT) / 2, (R_MAX_LON + R_MIN_LON) / 2],
    zoom_start=13,
    tiles=None,
)

# Define the corners of the rectangle
corners = [
    (R_MAX_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MIN_LON),
    (R_MIN_LAT, R_MAX_LON),
    (R_MAX_LAT, R_MAX_LON),
]

folium.Rectangle(
    bounds=[(R_MIN_LAT, R_MIN_LON), (R_MAX_LAT, R_MAX_LON)],
    color="black",
).add_to(map)

html_text = f"""
<div style="position: fixed; top: 50px; right: 50px; z-index:9999; font-size:14px;">
  &nbsp;<b>Res:</b> {RESOLUTION} km<br>
  &nbsp;<b>Layers:</b> {LAYERS}
</div>
"""

div_icon = folium.DivIcon(
    icon_size=(150, 36),
    icon_anchor=(0, 0),
    html=html_text,
)

# Choose a location for the upper right corner text
upper_right_location = [R_MAX_LAT - 0.001, R_MAX_LON - 0.003]

folium.Marker(location=upper_right_location, icon=div_icon).add_to(map)


folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr='Map data © <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',
    name="Transparent Tiles",
    opacity=0.2,
).add_to(
    map
)  # You can adjust opacity here

In [23]:
def read_coordinates(file_path):
    coordinates = []
    with open("../dataset/rome/output/" + file_path, "r") as file:
        count = 0
        for line in file:
            count += 1
            lat, lon = line.strip().split(", ")
            coordinates.append((float(lat), float(lon)))
        # print(f"Number of coordinates for {file}: {str(count)}")
    return coordinates


# Read the index file to get the list of files containing coordinates
with open(ROME_META_FILE, "r") as index_file:
    number_of_trajectories = 0
    for file_name in index_file:
        if number_of_trajectories == 2:
            break
        file_name = file_name.strip()  # Remove newline characters

        coordinates = read_coordinates(file_name)
        # Plot each coordinate on the map
        for lat, lon in coordinates:
            folium.CircleMarker(
                [lat, lon], radius=1, color="black", fill=False, fill_color="red"
            ).add_to(map)
        folium.PolyLine(coordinates, color="black",
                        weight=1, opacity=1).add_to(map)
        number_of_trajectories += 1

In [24]:
hashed_coordinates, grid = generate_grid_hash_similarity_coordinates(
    city="rome", res=RESOLUTION, layers=LAYERS, measure="dtw", size=50
)

for layer in grid:
    color = colors[layer]
    latitudes = grid[layer][0]
    longitudes = grid[layer][1]
    # print(f"Layer {layer} latitudes: {latitudes}")
    # print(f"Layer {layer} longitudes: {longitudes}")

    for lat in latitudes:
        for lon in longitudes:
            folium.CircleMarker(
                [lat, lon],
                radius=2,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=1,
            ).add_to(map)
    for lat in latitudes:
        polyline = [(lat, lon) for lon in longitudes]
        folium.PolyLine(polyline, color=color, weight=0.5).add_to(map)

    # Create polylines for longitude lines (vertical grid lines)
    for lon in longitudes:
        polyline = [(lat, lon) for lat in latitudes]
        folium.PolyLine(polyline, color=color, weight=0.5).add_to(map)

# print(f"\nNumber of hashed coordinates: {len(hashed_coordinates)}")
# print("Hashed coordinates:", hashed_coordinates)
number_of_hashed_trajectories = 0
for trajectory_key, trajectory_hash in hashed_coordinates.items():
    if number_of_hashed_trajectories == 2:
        break
    print(f"Trajectory hash: {trajectory_key} -> {trajectory_hash}")
    # print("len hash", len(trajectory_hash))
    layer = 0
    for hashed_trajectory_layer in trajectory_hash:
        print(f"Length of hashed coordinate in layer {layer}: {len(hashed_trajectory_layer)}" )
        for hashed_coordinate in hashed_trajectory_layer:
            folium.CircleMarker(
                hashed_coordinate, radius=10, color=colors[(layer)]
            ).add_to(map)
        layer += 1
    number_of_hashed_trajectories += 1

map

Trajectory hash: R_ATG -> [[[41.896954426323695, 12.440901499710737], [41.896954426323695, 12.443314548311903], [41.89514568285778, 12.445727596913068], [41.89514568285778, 12.448140645514234], [41.89514568285778, 12.4505536941154], [41.896954426323695, 12.4505536941154], [41.89876316978961, 12.4505536941154], [41.89876316978961, 12.452966742716566], [41.900571913255526, 12.45537979131773], [41.900571913255526, 12.457792839918897], [41.900571913255526, 12.460205888520063], [41.900571913255526, 12.462618937121228], [41.900571913255526, 12.465031985722394], [41.89876316978961, 12.467445034323559], [41.896954426323695, 12.469858082924725], [41.896954426323695, 12.47227113152589], [41.896954426323695, 12.474684180127056], [41.89876316978961, 12.474684180127056], [41.896954426323695, 12.47227113152589], [41.89876316978961, 12.47227113152589], [41.900571913255526, 12.47227113152589], [41.900571913255526, 12.474684180127056]], [[41.89713356935448, 12.441140494818665], [41.895324825888565, 12.